In [1]:
from httparse import RequestParser
from httptools import HttpRequestParser

In [2]:
small = b"GET /cookies/foo/bar/baz?a=1&b=2 HTTP/1.1\r\nHost: 127.0.0.1:8090\r\nConnection: keep-alive\r\nCache-Control: max-age=0\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8\r\nUser-Agent: Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.56 Safari/537.17\r\nAccept-Encoding: gzip,deflate,sdch\r\nAccept-Language: en-US,en;q=0.8\r\nAccept-Charset: ISO-8859-1,utf-8;q=0.7,*;q=0.3\r\nCookie: name=wookie\r\n\r\n"

In [3]:
import string
import random
import itertools
from typing import Iterable

CHARS = string.ascii_uppercase + string.digits + string.ascii_lowercase

CHUNK_SIZE = 65_536


def get_random_string(n: int) -> str:
    return ''.join(random.choices(CHARS, k=n))

headers = [
    (f"X-{get_random_string(15)}".encode(), get_random_string(1024))
    for _ in range(128)
]

headers_data = "\r\n".join([f"{name}: {val}" for name, val in headers])

large = f"GET /cookies/foo/bar/baz?a=1&b=2 HTTP/1.1\r\n{headers_data}\r\n\r\n".encode()

def grouper(n: int, data: bytes) -> Iterable[bytes]:
    it = iter(data)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield bytes(chunk)

In [4]:
def parse_until_complete_httparse(chunks: Iterable[bytes]):
    parser = RequestParser()
    buff = bytearray()
    for chunk in chunks:
        buff.extend(chunk)
        res = parser.parse(bytes(buff))
        if res is not None:
            return

class Proto:
    __slots__ = ("done")
    def __init__(self):
        self.done = False
    def on_headers_complete(self):
        self.done = True

def parse_until_complete_httptools(chunks: Iterable[bytes]):
    proto = Proto()
    parser = HttpRequestParser(proto)
    for chunk in chunks:
        parser.feed_data(chunk)
        if proto.done:
            return


SMALL_CHUNKS = list(grouper(CHUNK_SIZE, small))
LARGE_CHUNKS = list(grouper(CHUNK_SIZE, large))

In [5]:
%timeit parse_until_complete_httptools(SMALL_CHUNKS)

2.14 µs ± 29.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
%timeit parse_until_complete_httparse(SMALL_CHUNKS)

1.36 µs ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
%timeit parse_until_complete_httptools(LARGE_CHUNKS)

48 µs ± 82.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
%timeit parse_until_complete_httparse(LARGE_CHUNKS)

101 µs ± 424 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
